In [14]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import os

from PIL import Image
import numpy as np
import skimage
import matplotlib.pyplot as plt
import cv2
import time

ModuleNotFoundError: No module named 'cv2'

## In domain video accuracy

In [15]:
!pip install opencv

ERROR: Could not find a version that satisfies the requirement opencv (from versions: none)
ERROR: No matching distribution found for opencv


## Out of domain fine Tuning 

In [9]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import os

In [11]:
class SineLayer(nn.Module):
    def __init__(self, in_features, out_features, bias=True,
                 is_first=False, omega_0=30):
        super().__init__()
        self.omega_0 = omega_0
        self.is_first = is_first
        self.in_features = in_features
        self.linear = nn.Linear(in_features, out_features, bias=bias)
        dist = (1/self.in_features) if self.is_first else (np.sqrt(6/self.in_features))
        self.linear.weight.uniform_(-1*dist,dist)      
       
    def forward(self, input):
        return torch.sin(self.omega_0 * self.linear(input))
   
    
class Siren(nn.Module):
    def __init__(self, in_features, hidden_features, hidden_layers, out_features,first_omega_0=30, hidden_omega_0=30.):
        super().__init__()
        self.model = [SineLayer(in_features, hidden_features,is_first=True, omega_0=first_omega_0)]
        self.model += [SineLayer(hidden_features, hidden_features,is_first=False, omega_0=hidden_omega_0)]*hidden_layers
        final_linear = nn.Linear(hidden_features, out_features)
        with torch.no_grad():
            self.model += [final_linear.weight.uniform_(-np.sqrt(6 / hidden_features) / hidden_omega_0,np.sqrt(6 / hidden_features) / hidden_omega_0)]
        self.net = nn.Sequential(self.model)
    def forward(self, coords):
        coords = coords.clone().detach()
        output = self.net(coords)
        return output, coords        

In [12]:
def get_mgrid(sidelen, dim=2):
    tensors = tuple(dim * [torch.linspace(-1, 1, steps=sidelen)])
    mgrid = torch.stack(torch.meshgrid(*tensors), dim=-1)
    mgrid = mgrid.reshape(-1, dim)
    return mgrid

class ImageFitting(Dataset):
    def __init__(self):
        super().__init__()
        img  = torch.from_numpy(cv2.resize(cv2.imread("semifinalData/090920/102608_e_Cam1_NewNest_090920_im.jpg"),(256,256)))
        self.pixels = img.permute(1, 2, 0).view(-1, 1)
        self.coords = get_mgrid(sidelength, 2)
    def __len__(self):
        return 1
    def __getitem__(self, idx):    
        if idx > 0: raise IndexError
        return self.coords, self.pixels

In [13]:
im = ImageFitting()
dataloader = DataLoader(im, batch_size=1, pin_memory=True)

img_siren = Siren(in_features=2, out_features=1, hidden_features=256,hidden_layers=3)

total_steps = 500 # Since the whole image is our dataset, this just means 500 gradient descent steps.
steps_til_summary = 10

optim = torch.optim.Adam(lr=1e-4, params=img_siren.parameters())

model_input, ground_truth = next(iter(dataloader))
model_input, ground_truth = model_input, ground_truth

for step in range(total_steps):
    model_output, coords = img_siren(model_input)    
    loss = ((model_output - ground_truth)**2).mean()
    
    if not step % steps_til_summary:
        print("Step %d, Total loss %0.6f" % (step, loss))
        fig, axes = plt.subplots(1,1, figsize=(18,6))
        axes[0].imshow(model_output.view(256,256).detach().numpy())
        plt.show()

    optim.zero_grad()
    loss.backward()
    optim.step()

NameError: name 'cv2' is not defined